![Lifeboat](http://titanicwiki.com/wp-content/uploads/2012/03/lifeboat-around-Titanic.jpg)

In [1]:
sc

In [2]:
csv = '/Users/ltsai/Downloads/titanic3.csv'

In [3]:
from pyspark.sql.types import *
customSchema = StructType([StructField('pclass', StringType(), True),
                           StructField('survived', IntegerType(), True),
                           StructField('name', StringType(), True),
                           StructField('sex', StringType(), True),
                           StructField('age', StringType(), True),
                           StructField('sibsp', IntegerType(), True),
                           StructField('parch', IntegerType(), True),
                           StructField('ticket', StringType(), True),
                           StructField('fare', FloatType(), True),
                           StructField('cabin', StringType(), True),
                           StructField('embarked', StringType(), True),
                           StructField('boat', StringType(), True),
                           StructField('body', StringType(), True),
                           StructField('home.dest', StringType(), True)])
df = sqlContext.read.format('com.databricks.spark.csv'). \
        options(header='true', treatEmptyValuesAsNulls='true'). \
        load(csv, schema = customSchema)
df.registerTempTable("passengers")
sqlContext.cacheTable("passengers") 

In [19]:
train_df, test_df = df.select("survived", "pclass", "age").randomSplit([0.8, 0.2])
train_df.count(),test_df.count()

(1063, 246)

In [24]:
from pyspark.sql import Row
from pyspark.mllib.linalg import Vectors

ntrain = train_df.map(lambda x: Row(label = float(x[0]) \
         ,features = Vectors.dense(x[1:]))).toDF().cache() 

In [25]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter = 100, regParam = 0.1)
lr.fit(ntrain)

LogisticRegression_441285f6e8b232f72485